In [2]:
from langgraph.graph import StateGraph , START , END 
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from typing import Annotated , TypedDict 
import operator
from pydantic import BaseModel

In [3]:
llm = ChatOpenAI(
    temperature=0.2,
    base_url = "https://openrouter.ai/api/v1",
    openai_api_key = os.getenv("OPENROUTER_API_KEY"),
    model_name = "openrouter/polaris-alpha"
)

In [ ]:
class PostState(TypedDict):
    prompt:str
    image_urls:Annotated[list[str],operator.add]
    post_content:str
    post_title:str
    post_hashtags:list[str]

In [5]:
import json
with open("./variables.json",'r') as file:
    content = file.read()
variables = json.loads(content)

In [ ]:
class polish_output(BaseModel):
    original_prompt:str
    polished_prompt:str

def polish_prompt(state:PostState)->PostState:
    prompt = PromptTemplate.from_template(
        template = """
            The user is using an automated agent to post on {platform} about their platform herth . {about_herth} . 
            The user wants to create a post and they have provided a very casual prompt , your job is to tailor that prompt for {platform} to ensure that the post creation agent does a good job. 

            User's prompt:
            {prompt} 
        """
    )
    llm_structured = llm.with_structured_output(polish_output)
    prompt_value = prompt.invoke({"about_herth":variables["about_herth"],"platform":variables["platform"],"prompt":state["prompt"]})
    response = llm_structured.invoke(prompt_value)
    print(response)
    state['prompt'] = response.polished_prompt 
    return state

# response = polish_prompt({"prompt":"Post about our new posting feature in about 1 paragraph max"})

original_prompt='Post about our new posting feature in about 1 paragraph max' polished_prompt='Write a concise Reddit post (1 short paragraph, max 4–5 sentences) announcing Herth’s new posting feature. The tone should be warm, empowering, and community-focused, aligned with Herth’s mission to support women entrepreneurs through visibility, connection, and shared growth. Briefly explain what the new posting feature allows (e.g., sharing stories, showcasing businesses, finding collaborators, or offering/receiving mentorship), and highlight how it makes it easier for women founders and creatives on Herth to be discovered and supported. Avoid sounding overly salesy; emphasize authenticity, inclusivity, and the idea that when one woman grows, the whole community benefits. End with a light call-to-action inviting Redditors who are women entrepreneurs, or who support them, to check out Herth and share their work or feedback.'


In [ ]:
from typing import Annotated
from pydantic import BaseModel
from langchain.prompts import PromptTemplate

class generated_content_structure(BaseModel):
    content: str
    title: Annotated[str, "Generate a catchy, engaging title for the post"]
    hashtags: list[str]

def generate_content(state: dict) -> dict:
    prompt = PromptTemplate.from_template(
        """
        You are an automated agent that writes viral social media posts for {platform} about the platform Herth.

        About Herth:
        {about_herth}

        The user wants to create a post with maximum engagement.
        Write:
        1. The full post content.
        2. A catchy title for the post.
        3. 3–5 relevant hashtags.

        User’s prompt:
        {prompt}
        """
    )

    llm_structured = llm.with_structured_output(generated_content_structure)

    prompt_value = prompt.invoke({
        "about_herth": variables["about_herth"],
        "platform": variables["platform"],
        "prompt": state["prompt"]
    })

    response = llm_structured.invoke(prompt_value)

    print(response)
    state["post_content"] = response.content
    state["post_title"] = response.title
    state["post_hashtags"] = response.hashtags
    return state


# generate_content({"prompt":response['prompt']})

content='We just launched a new posting feature on Herth that makes it easier than ever for women founders and creatives to share their stories, showcase their work, find collaborators, and offer or receive mentorship in one supportive, community-driven space. Instead of getting lost in the noise, posts on Herth are designed to highlight real journeys, real impact, and real women doing the work—so the right people can discover, celebrate, and back them. It’s all about authentic visibility, reciprocity, and the belief that when one woman grows, our whole community rises with her. If you’re a woman entrepreneur (or someone who champions them), we’d love for you to check out Herth, share what you’re building, and tell us how we can make this space even more powerful for you.' title='Herth’s New Posting Feature: Real Women, Real Stories, Real Support' hashtags=['#WomenInBusiness', '#WomenFounders', '#CommunityOverCompetition', '#SupportWomenOwned', '#Herth']


{'prompt': 'Write a concise Reddit post (1 short paragraph, max 4–5 sentences) announcing Herth’s new posting feature. The tone should be warm, empowering, and community-focused, aligned with Herth’s mission to support women entrepreneurs through visibility, connection, and shared growth. Briefly explain what the new posting feature allows (e.g., sharing stories, showcasing businesses, finding collaborators, or offering/receiving mentorship), and highlight how it makes it easier for women founders and creatives on Herth to be discovered and supported. Avoid sounding overly salesy; emphasize authenticity, inclusivity, and the idea that when one woman grows, the whole community benefits. End with a light call-to-action inviting Redditors who are women entrepreneurs, or who support them, to check out Herth and share their work or feedback.',
 'post_content': 'We just launched a new posting feature on Herth that makes it easier than ever for women founders and creatives to share their stor

In [8]:
class check_generation:
    ai_generated:int
    authentic:int
    professional:int
    
def calculate_ai_generated(state:PostState)->PostState:
    return state
def calculate_authenticity(state:PostState)->PostState:
    return state
def calculate_professionalism(state:PostState)->PostState:
    return state

In [17]:
# making a state graph
checkpointer = MemorySaver()

graph = StateGraph(PostState)

graph.add_node("polish_prompt",polish_prompt)
graph.add_node("generate_content",generate_content)


# graph.add_node("calculate_ai_generated",calculate_ai_generated)
# graph.add_node("calculate_authenticity",calculate_authenticity)
# graph.add_node("calculate_professionalism",calculate_professionalism)

graph.add_edge(START,"polish_prompt")
graph.add_edge("polish_prompt","generate_content")
graph.add_edge("generate_content",END)

workflow = graph.compile(checkpointer = checkpointer)


In [22]:
config = {"configurable":{"thread_id":1}}
final_state = workflow.invoke({"prompt":"Make a post about our new upcoming platform"},config = config)

original_prompt='Make a post about our new upcoming platform' polished_prompt='Create a Reddit post announcing our upcoming platform, Herth, in a clear, authentic, and community-focused tone suitable for entrepreneurial and women-in-business subreddits (e.g., r/Entrepreneur, r/womeninbiz, r/startups). The post should:\n\n- Start with a concise, engaging hook that highlights the problem: how many women entrepreneurs struggle with visibility, access to networks, and authentic support.\n- Introduce Herth as a community-driven platform built to empower women entrepreneurs to grow, connect, and thrive.\n- Clearly explain our mission: to close the gap between ambition and opportunity by providing visibility, support, and tools that help women turn potential into performance.\n- Emphasize key values and differentiators:\n  - Herth transforms isolated business journeys into shared experiences through meaningful networks and practical mentorship.\n  - We focus on sustainability, inclusivity, an

In [23]:
import praw
load_dotenv() 
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    username=os.getenv("USERNAME"),
    password=os.getenv("PASSWORD"),
    user_agent=os.getenv("USER_AGENT")
)

print("Logged in as:", reddit.user.me())

# ---- POST TO SUBREDDIT ----
subreddit = reddit.subreddit("test")  # change to your target subreddit
# title = "Testing Reddit API Post 🧠"
# body = "This post was made automatically via Python using PRAW."

title = final_state['post_title']
content = final_state['post_content']

subreddit.submit(title=title, selftext=content)
print("Post submitted!")

Logged in as: brehimsodonee
Post submitted!
